# Introduction to LLM Engineering

## Installing ollama

URL: https://ollama.com/download 

### How to pull a model locally?
!ollama pull llama3.2

ollama commands:
1. ollama --version
2. ollama list
3. ollama ps
4. ollama show model_name

In [15]:
!ollama show llama3.2

  Model
    architecture        llama     
    parameters          3.2B      
    context length      131072    
    embedding length    3072      
    quantization        Q4_K_M    

  Capabilities
    completion    
    tools         

  Parameters
    stop    "<|start_header_id|>"    
    stop    "<|end_header_id|>"      
    stop    "<|eot_id|>"             

  License
    LLAMA 3.2 COMMUNITY LICENSE AGREEMENT                 
    Llama 3.2 Version Release Date: September 25, 2024    
    ...                                                   



## Connect to ollama

In [1]:
from openai import OpenAI
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = openai.chat.completions.create(
    model = MODEL,
    messages=[{"role": "user", "content": "What is your purpose?"}]
)
print(response.choices[0].message.content)

I’m a large language model. When you ask me a question or provide me with a prompt, I analyze what you say and generate a response that is relevant and accurate. I'm constantly learning and improving, so over time I'll be even better at assisting you. Is there anything I can help you with?


# Weather report in summary for your current location

In [25]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [26]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Weather:
    def __init__(self, url):
        """
        This object will be used to fetch weather data from a given URL
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        self.text = soup.body.get_text(separator="\n", strip=True)

## Prompts - System and User

In [27]:
system_prompts = "You are an helpful assitant that analyzer a weather website contant and provide short summary. Response in markdown."

In [28]:
def user_prompt(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\n The content of this website is as follows: \
                    Please provide a summary of this website in markdown. \
                    Properly state the current time, current weather, and current UV index in a separate paragraph in listing format. \
                    Create a pie chart to show the above information. \n\n"
    user_prompt += website.text
    return user_prompt

In [29]:
def message_for(website):
    return [
        {"role": "system", "content": system_prompts},
        {"role": "user", "content": user_prompt(website)}
    ]

In [30]:
def summarize(url):
    website = Weather(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = message_for(website)
    )
    return response.choices[0].message.content

In [31]:
summarize("https://www.theweathernetwork.com/en/city/ca/alberta/calgary/current")

"**Summary**\nThe Calgary, AB Current Weather website provides current weather conditions, forecasts, and news. The website is updated regularly with the latest weather information, including temperature, humidity, wind, pressure, and precipitation.\n\n### Important Information\n\n* **Current Time**: Not available on this website\n* **Current Weather**:\n\t+ Temperature: 25°C\n\t+ Conditions: Partly cloudy\n\t+ Feels like: 27°C\n* **UV Index**: Not available on this website\n\nNote: The UV index is not provided on this website, so I couldn't include it in the above format. Here's a pie chart representing the current weather information:\n```markdown\n Pie Chart: Calgary, AB Current Weather\n=====================================================\n\n       Temperature     | 25°C ( 77°F )\n-------------------\n       Conditions     | Partly cloudy\n-------------------\n         Feels like   | 27°C ( 81°F )\n```\nPlease note that this pie chart is a simplified representation and not visuall